In [ ]:
! gdown 1yEjicXasMGFaALQobQWgsSxQgLEtxruC
! unzip /content/ThaiDepression.zip

Downloading...
From: https://drive.google.com/uc?id=1yEjicXasMGFaALQobQWgsSxQgLEtxruC
To: /content/ThaiDepression.zip
100% 41.1k/41.1k [00:00<00:00, 50.2MB/s]
Archive:  /content/ThaiDepression.zip
  inflating: ThaiDepression/AHB_01.csv  
  inflating: ThaiDepression/AHB_02.csv  
  inflating: ThaiDepression/AHB_03.csv  
  inflating: ThaiDepression/AHB_04.csv  
  inflating: ThaiDepression/AHB_05.csv  
  inflating: ThaiDepression/AHB_06.csv  
  inflating: ThaiDepression/AHB_07.csv  
  inflating: ThaiDepression/AHB_08.csv  
  inflating: ThaiDepression/AHB_09.csv  
  inflating: ThaiDepression/AHB_10.csv  
  inflating: ThaiDepression/AHB_11.csv  
  inflating: ThaiDepression/AHB_12.csv  
  inflating: ThaiDepression/AHB_13.csv  
  inflating: ThaiDepression/All_AHB.csv  


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12481624609388254703
xla_global_id: -1
]


In [ ]:
import pandas as pd
import numpy as np
import os
from numpy import array
from math import sqrt
from numpy import mean
from pandas import DataFrame
from pandas import concat
from pandas import read_csv
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler
from keras.preprocessing import sequence
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, GRU, BatchNormalization
from pandas import read_csv
from keras import Model
from keras.layers import Layer
import keras.backend as K
from keras.layers import Input, SimpleRNN
import json
import psutil 
from datetime import datetime, timezone, timedelta

In [ ]:
file_list = os.listdir('ThaiDepression')
file_list.sort()
df_list = []
for i in range(14):
    df_list.append(pd.read_csv('ThaiDepression/' + file_list[i], index_col=0))

In [ ]:
df_list

[         Chiang_Rai    Nan  Phayao  Phrae  Chiang_Mai  Mae_Hong_Son  Lampang  \
 2016-10       10383   5315    4266   4057       15408          2002     9011   
 2016-11       10383   5315    4265   4057       15406          2002     9011   
 2016-12       10382   5315    4265   4057       15404          2002     9011   
 2017-01       10382   5655    4265   4057       15393          2002     9011   
 2017-02       10382   5655    4264   4057       15392          2002     9011   
 ...             ...    ...     ...    ...         ...           ...      ...   
 2022-03       25223   9711   10788   7988       47292          6924    17444   
 2022-04       25456   9781   10932   8095       48020          7004    17606   
 2022-05       25619   9833   10977   8122       48155          7026    17703   
 2022-06       25836   9884   11069   8161       48616          7090    17783   
 2022-07       26174  10019   11163   8250       49096          7169    17951   
 
          Lamphun   AHB_1 

In [ ]:
# split a univariate sequence into samples
def uni_split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


    # split a multivariate sequence into samples
def multi_split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


def get_scaler(scaler):
    scalers = {
        "minmax": MinMaxScaler,
        "standard": StandardScaler,
        "maxabs": MaxAbsScaler,
        "robust": RobustScaler,
    }
    return scalers.get(scaler.lower())()

In [ ]:
%matplotlib inline
# define scope of configs
AHB_list = [8, 9]
BiLSTM_hidden_units = [8, 16, 32, 64, 128]
n_layer = [1, 2, 3]
n_batch = [2, 4, 8, 16, 32, 64, 128]
n_steps = [5]
patience_list = [3, 5, 7, 10]
mul_uni = ['uni','mul']
dropouts = [0.2, 0.3, 0.5]


timezone_offset = 7.0  # Bangkok Time (UTC+07:00)
tzinfo = timezone(timedelta(hours=timezone_offset))
count = 1
performance_json = []
for g in AHB_list:
    for h in patience_list:
        for a in n_steps:
            for b in dropouts:
                for c in BiLSTM_hidden_units:
                    for d in n_layer:
                        for e in n_batch:
                            for f in mul_uni:

                                print('PROGRESS: [ ' + str(count) + ' / ' + str(len(dropouts)*len(AHB_list)*len(n_steps)*len(BiLSTM_hidden_units)*len(n_layer)*len(n_batch)*len(patience_list)*len(mul_uni)) + ' ]')
                                
                                if f == 'uni':
                                    if g == 13:
                                        raw_seq = list(df_list[g]['Country_Level'])
                                        ahb = 'Country_Level'
                                    else:    
                                        raw_seq = list(df_list[g]['AHB_'+str(g+1)])
                                        ahb = 'AHB_'+str(g+1)
                                    n_features = 1
                                    scaler = get_scaler('minmax')
                                    raw_seq_arr = scaler.fit_transform(np.array(raw_seq).reshape(len(raw_seq),-1))
                                    X, y = uni_split_sequence(raw_seq_arr, a)
                                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, train_size=0.9, shuffle=False, stratify=None)
                                    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
                                    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))
                                if f == 'mul':
                                    if g == 13:
                                        raw_seq = list(df_list[13].drop(df_list[13].columns[:77], axis=1).values)
                                        ahb = 'Country_Level'
                                        scaler_y = get_scaler('minmax')
                                        scaler_y.fit(np.array(df_list[13]['Country_Level']).reshape(len(df_list[13]['Country_Level']),-1))
                                    else:    
                                        raw_seq = list(df_list[g].drop(['Country_Level'], axis=1).values)
                                        ahb = 'AHB_'+str(g+1)
                                        scaler_y = get_scaler('minmax')
                                        scaler_y.fit(np.array(df_list[g]['AHB_'+str(g+1)]).reshape(len(df_list[13]['AHB_'+str(g+1)]),-1))
                                    n_features = len(raw_seq[0])-1
                                    scaler = get_scaler('minmax')
                                    raw_seq_arr = scaler.fit_transform(np.array(raw_seq))
                                    X, y = multi_split_sequences(raw_seq_arr, a)
                                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, train_size=0.9, shuffle=False, stratify=None)
                                    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
                                    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))


                                callback = EarlyStopping(monitor='loss', patience=h)
                                model = Sequential()
                                if d == 1:
                                    model.add(Bidirectional(LSTM(units=c, return_sequences=False, input_shape=(a,n_features), activation='tanh')))
                                    model.add(Dropout(b))
                                if d == 2:
                                    model.add(Bidirectional(LSTM(units=c, return_sequences=True, input_shape=(a,n_features), activation='tanh')))
                                    model.add(Dropout(b))
                                    model.add(Bidirectional(LSTM(units=c, return_sequences=False, activation='tanh')))
                                    model.add(Dropout(b))
                                if d == 3:
                                    model.add(Bidirectional(LSTM(units=c, return_sequences=True, input_shape=(a,n_features), activation='tanh')))
                                    model.add(Dropout(b))
                                    model.add(Bidirectional(LSTM(units=c, return_sequences=True, input_shape=(a,n_features), activation='tanh')))
                                    model.add(Dropout(b))
                                    model.add(Bidirectional(LSTM(units=c, return_sequences=False, activation='tanh')))
                                    model.add(Dropout(b))
                                model.add(Dense(units=1, kernel_initializer='he_uniform', activation='linear'))
                                model.compile(loss='mse', optimizer='adam')  
                                history = model.fit(X_train, y_train, epochs=2000, batch_size=e, verbose=0, callbacks=[callback])
                                last_epoch = len(history.history['loss']) 

                                yhat = model.predict(X_test, verbose=0)
                                yhat = yhat.reshape(yhat.shape[0],-1)
                                mae = mean_absolute_error(y_test, yhat)
                                mse = mean_squared_error(y_test, yhat)
                                rmse = mean_squared_error(y_test, yhat, squared = False)
                                mape = mean_absolute_percentage_error(y_test, yhat)
                                
                                if f == 'uni':
                                    yhat_inverse = scaler.inverse_transform(yhat)
                                    y_test_inverse = scaler.inverse_transform(y_test)
                                if f == 'mul':
                                    yhat_inverse = scaler_y.inverse_transform(yhat)
                                    y_test_inverse = scaler_y.inverse_transform(y_test.reshape(len(y_test),-1))


                                mae_inverse = mean_absolute_error(y_test_inverse, yhat_inverse)
                                mse_inverse = mean_squared_error(y_test_inverse, yhat_inverse)
                                rmse_inverse = mean_squared_error(y_test_inverse, yhat_inverse, squared = False)
                                mape_inverse = mean_absolute_percentage_error(y_test_inverse, yhat_inverse)

                                performance_json.append({'model': 'BiLSTM', 'AHB_number': ahb, 'Multi_Uni': f, \
                                                         'n_layer': d, 'BiLSTM_hidden_units': c, 'dropouts': b,'n_batch': e,\
                                                         'n_steps': a, 'last_epoch': last_epoch, 'patience_list': h, 'MAE_normalize': float(mae), \
                                                         'MSE_normalize': float(mse), 'RMSE_normalize': float(rmse), 'MAPE_normalize': float(mape), \
                                                         'MAE': float(mae_inverse), 'MSE': float(mse_inverse), 'RMSE': float(rmse_inverse), 'MAPE': float(mape_inverse)})
                                

                                json_formatted_str = json.dumps(performance_json[-1], indent=2)
                                print(json_formatted_str)
                                print('Percent CPU Usage: ' + str(psutil.cpu_percent()))
                                print('Percent Ram Usage: ' + str(psutil.virtual_memory()[2]))
                                print(datetime.now(tzinfo))

                                count = count + 1

PROGRESS: [ 1 / 5040 ]
{
  "model": "BiLSTM",
  "AHB_number": "AHB_9",
  "Multi_Uni": "uni",
  "n_layer": 1,
  "BiLSTM_hidden_units": 8,
  "dropouts": 0.2,
  "n_batch": 2,
  "n_steps": 5,
  "last_epoch": 23,
  "patience_list": 3,
  "MAE_normalize": 0.010210975833636202,
  "MSE_normalize": 0.00013631461148206896,
  "RMSE_normalize": 0.011675384853702637,
  "MAPE_normalize": 0.010983552051028704,
  "MAE": 797.1696428571429,
  "MSE": 830824.8589564732,
  "RMSE": 911.4959456610178,
  "MAPE": 0.0055302953151476954
}
Percent CPU Usage: 39.2
Percent Ram Usage: 10.5
2022-10-11 13:50:19.966058+07:00
PROGRESS: [ 2 / 5040 ]
{
  "model": "BiLSTM",
  "AHB_number": "AHB_9",
  "Multi_Uni": "mul",
  "n_layer": 1,
  "BiLSTM_hidden_units": 8,
  "dropouts": 0.2,
  "n_batch": 2,
  "n_steps": 5,
  "last_epoch": 13,
  "patience_list": 3,
  "MAE_normalize": 0.0633042179693336,
  "MSE_normalize": 0.004030849319489673,
  "RMSE_normalize": 0.06348897006165459,
  "MAPE_normalize": 0.06671012737449453,
  "MAE": 4

{
  "model": "BiLSTM",
  "AHB_number": "AHB_9",
  "Multi_Uni": "uni",
  "n_layer": 1,
  "BiLSTM_hidden_units": 8,
  "dropouts": 0.2,
  "n_batch": 8,
  "n_steps": 5,
  "last_epoch": 12,
  "patience_list": 3,
  "MAE_normalize": 0.18135825881002748,
  "MSE_normalize": 0.03327767147283142,
  "RMSE_normalize": 0.18242168586226645,
  "MAPE_normalize": 0.19074290543275887,
  "MAE": 14158.64174107143,
  "MSE": 202824976.75878033,
  "RMSE": 14241.663412634787,
  "MAPE": 0.09710944083343788
}
Percent CPU Usage: 51.9
Percent Ram Usage: 11.3
2022-10-11 13:50:43.408092+07:00
PROGRESS: [ 6 / 5040 ]


Streaming output truncated to the last 5000 lines.
  "MAE": 2146.5379464285716,
  "MSE": 4637412.608607701,
  "RMSE": 2153.4652559555498,
  "MAPE": 0.021042711983772094
}
Percent CPU Usage: 69.9
Percent Ram Usage: 14.3
2022-10-12 06:51:57.222059+07:00
PROGRESS: [ 4833 / 5040 ]
{
  "model": "BiLSTM",
  "AHB_number": "AHB_10",
  "Multi_Uni": "uni",
  "n_layer": 1,
  "BiLSTM_hidden_units": 8,
  "dropouts": 0.5,
  "n_batch": 4,
  "n_steps": 5,
  "last_epoch": 37,
  "patience_list": 10,
  "MAE_normalize": 0.08734817598689874,
  "MSE_normalize": 0.007655644513044031,
  "RMSE_normalize": 0.08749654000612841,
  "MAPE_normalize": 0.0898669321000568,
  "MAE": 4969.849330357143,
  "MSE": 24783379.009024482,
  "RMSE": 4978.290771843735,
  "MAPE": 0.0487227696191385
}
Percent CPU Usage: 62.2
Percent Ram Usage: 14.4
2022-10-12 06:52:04.205009+07:00
PROGRESS: [ 4834 / 5040 ]
{
  "model": "BiLSTM",
  "AHB_number": "AHB_10",
  "Multi_Uni": "mul",
  "n_layer": 1,
  "BiLSTM_hidden_units": 8,
  "dropouts"

In [ ]:
name = "all_performance_BiLSTM_5nsteps_AHB9-10"
json.dump(performance_json, open(name + ".json","w"))
df_json = pd.read_json('/content/' + name + '.json')
df_json.to_csv('/content/' + name + '.csv', index=False)
#! cp /content/all_performance_BiLSTM_5nsteps_AHB9-10.json /content/drive/MyDrive/Depression
! cp /content/all_performance_BiLSTM_5nsteps_AHB9-10.csv /content/drive/MyDrive/Depression

In [ ]:
df_json

,model,AHB_number,Multi_Uni,n_layer,BiLSTM_hidden_units,dropouts,n_batch,n_steps,last_epoch,patience_list,MAE_normalize,MSE_normalize,RMSE_normalize,MAPE_normalize,MAE,MSE,RMSE,MAPE
0,BiLSTM,AHB_9,uni,1,8,0.2,2,5,23,3,0.010211,0.000136,0.011675,0.010984,797.169643,8.308249e+05,911.495946,0.005530
1,BiLSTM,AHB_9,mul,1,8,0.2,2,5,13,3,0.063304,0.004031,0.063489,0.066710,4942.158482,2.456771e+07,4956.582080,0.033931
2,BiLSTM,AHB_9,uni,1,8,0.2,4,5,10,3,0.168488,0.028736,0.169516,0.177193,13153.890625,1.751423e+08,13234.132819,0.090215
3,BiLSTM,AHB_9,mul,1,8,0.2,4,5,11,3,0.062332,0.003944,0.062800,0.065534,4866.274554,2.403744e+07,4902.799651,0.033370
4,BiLSTM,AHB_9,uni,1,8,0.2,8,5,12,3,0.181358,0.033278,0.182422,0.190743,14158.641741,2.028250e+08,14241.663413,0.097109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5035,BiLSTM,AHB_10,mul,3,128,0.5,32,5,38,10,0.042114,0.001787,0.042279,0.043307,2396.152902,5.786568e+06,2405.528669,0.023485
5036,BiLSTM,AHB_10,uni,3,128,0.5,64,5,49,10,0.031762,0.001014,0.031849,0.032731,1807.183036,3.283757e+06,1812.114073,0.017732
5037,BiLSTM,AHB_10,mul,3,128,0.5,64,5,25,10,0.159341,0.025421,0.159439,0.164003,9066.051339,8.229443e+07,9071.627736,0.088900
5038,BiLSTM,AHB_10,uni,3,128,0.5,128,5,44,10,0.058493,0.003432,0.058579,0.060194,3328.069196,1.110875e+07,3332.978713,0.032632
